**Tugas Transfer Learning**  
<br>
<ul style="list-style-type:none;">
    <li>Nama: Rachmat Dwi Putra </li>
    <li>NIM : 1301180201 </li>
</ul>

# Load Dataset CIFAR-10

In [1]:
from tensorflow.keras.datasets import cifar10 # import dataset CIFAR-10 dari modul keras
(x_train, y_train), (x_test, y_test) = cifar10.load_data() # load dataset CIFAR-10 dan membaginya ke dalam data train dan data test

# Analisis Performa Awal Model VGG-16 dan ResNet-50

## Menggunakan Model VGG-16

Model menggunakan pre-trained VGG-16, 128 layer dense neuron, optimizer <code>'adam'</code>, 5 epoch, dan 32 batch size

In [2]:
#model pre-trained
#Gunakan model seperti VGG16 atau ResNet50

from tensorflow.keras.applications import VGG16 # import model pre-trained VGG-16 yang telah dilath pada dataset ImageNet
from tensorflow.keras.models import Model # import kelas Model untuk membuat arsitektur model kustom
from tensorflow.keras.layers import Dense, Flatten # impor layer Dense untuk menambahkan lapisan fully connected dan Flatten untuk meratakan tensor input

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3)) # Membuat model VGG16 tanpa bagian atas '(include_top=False)', sehingga hanya digunakan sebagai feature extractor dengan input gambar berukuran 32x32x3
base_model.trainable = False  # Membekukan lapisan pre-trained VGG16 agar tidak diperbarui selama pelatihan

x = Flatten()(base_model.output) # Menambahkan layer Flatten untuk meratakan keluaran dari model dasar
x = Dense(128, activation='relu')(x) # Menambahkan lapisan fully connected dengan 128 neuron dan fungsi aktivasi ReLU
output = Dense(10, activation='softmax')(x) # Menambahkan lapisan keluaran dengan 10 neuron (sesuai jumlah kelas pada CIFAR-10) menggunakan aktivasi softmax.

model = Model(inputs=base_model.input, outputs=output) # Membuat model akhir dengan input dari model dasar dan output yang disesuaikan
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy']) # Mengompilasi model menggunakan optimizer Adam, fungsi loss sparse categorical cross-entropy, dan metrik akurasi.
model.summary() # Menampilkan ringkasan arsitektur model.

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 16, 16, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 16, 16, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 8, 8, 128)         0     

In [3]:
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=5, batch_size=32) # Melatih model menggunakan data latih, memvalidasinya dengan data uji, selama 5 epoch dengan ukuran batch 32.

Epoch 1/5
1563/1563 [==============================] - 226s 144ms/step - loss: 2.2097 - accuracy: 0.4634 - val_loss: 1.3478 - val_accuracy: 0.5387
Epoch 2/5
1563/1563 [==============================] - 250s 160ms/step - loss: 1.2139 - accuracy: 0.5822 - val_loss: 1.2377 - val_accuracy: 0.5839
Epoch 3/5
1563/1563 [==============================] - 280s 179ms/step - loss: 1.1265 - accuracy: 0.6107 - val_loss: 1.2117 - val_accuracy: 0.5894
Epoch 4/5
1563/1563 [==============================] - 252s 161ms/step - loss: 1.0811 - accuracy: 0.6278 - val_loss: 1.2311 - val_accuracy: 0.5871
Epoch 5/5
1563/1563 [==============================] - 253s 162ms/step - loss: 1.0371 - accuracy: 0.6414 - val_loss: 1.2367 - val_accuracy: 0.5933


## Menggunakan Model Resnet-50

Model menggunakan pre-trained ResNet-50, 128 layer dense neuron, optimizer <code>'adam'</code>, 5 epoch, dan 32 batch size

In [4]:
from tensorflow.keras.applications import ResNet50

# Membuat base model ResNet50
base_model_2 = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3)) # import model pre-trained ResNet-50 yang telah dilath pada dataset ImageNet
base_model_2.trainable = False  # Membekukan lapisan pre-trained ResNet-50

# Menambahkan lapisan baru setelah base model
x_2 = Flatten()(base_model_2.output)  # Flatten hasil feature extractor
x_2 = Dense(128, activation='relu')(x_2)  # Layer Dense dengan 128 neuron
output_2 = Dense(10, activation='softmax')(x_2)  # Layer output dengan 10 kelas (CIFAR-10)

# Membuat model akhir
model_2 = Model(inputs=base_model_2.input, outputs=output_2)

# Compile model
model_2.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Menampilkan ringkasan model
model_2.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 38, 38, 3)    0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 16, 16, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 16, 16, 64)   256         ['conv1_conv[0][0]']             
                                                                                            

                                                                                                  
 conv2_block3_2_bn (BatchNormal  (None, 8, 8, 64)    256         ['conv2_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv2_block3_2_relu (Activatio  (None, 8, 8, 64)    0           ['conv2_block3_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv2_block3_3_conv (Conv2D)   (None, 8, 8, 256)    16640       ['conv2_block3_2_relu[0][0]']    
                                                                                                  
 conv2_block3_3_bn (BatchNormal  (None, 8, 8, 256)   1024        ['conv2_block3_3_conv[0][0]']    
 ization) 

                                                                                                  
 conv3_block3_2_bn (BatchNormal  (None, 4, 4, 128)   512         ['conv3_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block3_2_relu (Activatio  (None, 4, 4, 128)   0           ['conv3_block3_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block3_3_conv (Conv2D)   (None, 4, 4, 512)    66048       ['conv3_block3_2_relu[0][0]']    
                                                                                                  
 conv3_block3_3_bn (BatchNormal  (None, 4, 4, 512)   2048        ['conv3_block3_3_conv[0][0]']    
 ization) 

                                                                                                  
 conv4_block2_2_bn (BatchNormal  (None, 2, 2, 256)   1024        ['conv4_block2_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block2_2_relu (Activatio  (None, 2, 2, 256)   0           ['conv4_block2_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block2_3_conv (Conv2D)   (None, 2, 2, 1024)   263168      ['conv4_block2_2_relu[0][0]']    
                                                                                                  
 conv4_block2_3_bn (BatchNormal  (None, 2, 2, 1024)  4096        ['conv4_block2_3_conv[0][0]']    
 ization) 

 n)                                                                                               
                                                                                                  
 conv4_block5_3_conv (Conv2D)   (None, 2, 2, 1024)   263168      ['conv4_block5_2_relu[0][0]']    
                                                                                                  
 conv4_block5_3_bn (BatchNormal  (None, 2, 2, 1024)  4096        ['conv4_block5_3_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block5_add (Add)         (None, 2, 2, 1024)   0           ['conv4_block4_out[0][0]',       
                                                                  'conv4_block5_3_bn[0][0]']      
                                                                                                  
 conv4_blo

 n)                                                                                               
                                                                                                  
 conv5_block2_3_conv (Conv2D)   (None, 1, 1, 2048)   1050624     ['conv5_block2_2_relu[0][0]']    
                                                                                                  
 conv5_block2_3_bn (BatchNormal  (None, 1, 1, 2048)  8192        ['conv5_block2_3_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv5_block2_add (Add)         (None, 1, 1, 2048)   0           ['conv5_block1_out[0][0]',       
                                                                  'conv5_block2_3_bn[0][0]']      
                                                                                                  
 conv5_blo

In [5]:
model_2.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=5, batch_size=32)

Epoch 1/5
1563/1563 [==============================] - 175s 110ms/step - loss: 1.2969 - accuracy: 0.5496 - val_loss: 1.1780 - val_accuracy: 0.5859
Epoch 2/5
1563/1563 [==============================] - 668s 428ms/step - loss: 1.0709 - accuracy: 0.6222 - val_loss: 1.1225 - val_accuracy: 0.6092
Epoch 3/5
1563/1563 [==============================] - 190s 122ms/step - loss: 0.9928 - accuracy: 0.6494 - val_loss: 1.1490 - val_accuracy: 0.6086
Epoch 4/5
1563/1563 [==============================] - 163s 104ms/step - loss: 0.9336 - accuracy: 0.6701 - val_loss: 1.1577 - val_accuracy: 0.6029
Epoch 5/5
1563/1563 [==============================] - 6447s 4s/step - loss: 0.8846 - accuracy: 0.6867 - val_loss: 1.1767 - val_accuracy: 0.6056


## Hasil Analisa Performa Awal Kedua Model

Kesimpulan:

<ul>
    <li>Nilai <code>'loss'</code> yang makin menurun/rendah nilainya dari epoch ke epoch selanjutnya menunjukkan kedua model semakin baik dalam memprediksi data training</li>
    <li>Nilai <code>'accuracy'</code> yang makin menaik/tinggi nilainya dari epoch ke epoch selanjutnya menunjukkan kedua model mampu memprediksi label data training dengan baik</li>
    <li>Nilai <code>'val_loss'</code> yang nilainya berubah naik turun secara tidak konsisten dari epoch ke epoch selanjutnya menunjukkan kedua model belum mampu menggeneralisasi data validation dengan baik</li>
    <li>Nilai <code>'val_accuracy'</code> yang semakin tinggi nilainya dari epoch ke epoch selanjutnya menunjukkan kedua model model mampu memprediksi data validation dengan baik, sehingga model memiliki kemampuan generalisasi yang baik</li>
    <li>Namun nilai <code>'val_loss'</code> lebih tinggi daripada nilai <code>'loss'</code> dan nilai <code>'val_accuracy'</code> lebih rendah dibandingkan nilai <code>'accuracy'</code>, ada kemungkinan kedua model mengalami overfitting pada data training</li>
    <li>Secara keseluruhan, pada model base ResNet-50, baik nilai <code>'loss'</code>, <code>'accuracy'</code>, <code>'val_loss'</code>, dan <code>'val_accuracy'</code>, semua nilainya ini lebih tinggi dibandingkan dengan semua nilai pada model base VGG-16</li>
</ul>

# Analisis Eksperimen Mengubah Berbagai Variabel Pada Model VGG-16 dan ResNet-50

## Eksperimen Mengubah Variabel Jumlah Layer Dense

### Eksperimen 1 (Model Base VGG-16)

Model menggunakan pre-trained VGG-16, 256 layer dense neuron, optimizer <code>'adam'</code>, 5 epoch, dan 32 batch size

In [6]:
# Membuat base model VGG-16
base_model_3 = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3)) # import model pre-trained VGG-16 yang telah dilath pada dataset ImageNet
base_model_3.trainable = False  # Membekukan lapisan pre-trained VGG-16

# Menambahkan lapisan baru setelah base model
x_3 = Flatten()(base_model_3.output)  # Flatten hasil feature extractor
x_3 = Dense(256, activation='relu')(x_3)  # Layer Dense dengan 256 neuron
output_3 = Dense(10, activation='softmax')(x_3)  # Layer output dengan 10 kelas (CIFAR-10)

# Membuat model akhir
model_3 = Model(inputs=base_model_3.input, outputs=output_3)

# Compile model
model_3.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Menampilkan ringkasan model
model_3.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 16, 16, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 16, 16, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 8, 8, 128)         0   

In [7]:
model_3.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=5, batch_size=32)

Epoch 1/5
1563/1563 [==============================] - 268s 170ms/step - loss: 2.3837 - accuracy: 0.4930 - val_loss: 1.3145 - val_accuracy: 0.5570
Epoch 2/5
1563/1563 [==============================] - 247s 158ms/step - loss: 1.1695 - accuracy: 0.5963 - val_loss: 1.2355 - val_accuracy: 0.5880
Epoch 3/5
1563/1563 [==============================] - 930s 595ms/step - loss: 1.0738 - accuracy: 0.6280 - val_loss: 1.2257 - val_accuracy: 0.5972
Epoch 4/5
1563/1563 [==============================] - 801s 513ms/step - loss: 1.0224 - accuracy: 0.6461 - val_loss: 1.2440 - val_accuracy: 0.5912
Epoch 5/5
1563/1563 [==============================] - 876s 561ms/step - loss: 0.9714 - accuracy: 0.6663 - val_loss: 1.2541 - val_accuracy: 0.5985


### Eksperimen 2 (Model Base ResNet-50)

Model menggunakan pre-trained ResNet-50, 256 layer dense neuron, optimizer <code>'adam'</code>, 5 epoch, dan 32 batch size

In [8]:
# Membuat base model Resnet-50
base_model_4 = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3)) # import model pre-trained Resnet-50 yang telah dilath pada dataset ImageNet
base_model_4.trainable = False  # Membekukan lapisan pre-trained Resnet-50

# Menambahkan lapisan baru setelah base model
x_4 = Flatten()(base_model_4.output)  # Flatten hasil feature extractor
x_4 = Dense(256, activation='relu')(x_4)  # Layer Dense dengan 256 neuron
output_4 = Dense(10, activation='softmax')(x_4)  # Layer output dengan 10 kelas (CIFAR-10)

# Membuat model akhir
model_4 = Model(inputs=base_model_4.input, outputs=output_4)

# Compile model
model_4.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Menampilkan ringkasan model
model_4.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 38, 38, 3)    0           ['input_4[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 16, 16, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 16, 16, 64)   256         ['conv1_conv[0][0]']             
                                                                                            

                                                                                                  
 conv2_block3_2_bn (BatchNormal  (None, 8, 8, 64)    256         ['conv2_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv2_block3_2_relu (Activatio  (None, 8, 8, 64)    0           ['conv2_block3_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv2_block3_3_conv (Conv2D)   (None, 8, 8, 256)    16640       ['conv2_block3_2_relu[0][0]']    
                                                                                                  
 conv2_block3_3_bn (BatchNormal  (None, 8, 8, 256)   1024        ['conv2_block3_3_conv[0][0]']    
 ization) 

                                                                                                  
 conv3_block3_2_bn (BatchNormal  (None, 4, 4, 128)   512         ['conv3_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block3_2_relu (Activatio  (None, 4, 4, 128)   0           ['conv3_block3_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block3_3_conv (Conv2D)   (None, 4, 4, 512)    66048       ['conv3_block3_2_relu[0][0]']    
                                                                                                  
 conv3_block3_3_bn (BatchNormal  (None, 4, 4, 512)   2048        ['conv3_block3_3_conv[0][0]']    
 ization) 

                                                                                                  
 conv4_block2_2_bn (BatchNormal  (None, 2, 2, 256)   1024        ['conv4_block2_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block2_2_relu (Activatio  (None, 2, 2, 256)   0           ['conv4_block2_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block2_3_conv (Conv2D)   (None, 2, 2, 1024)   263168      ['conv4_block2_2_relu[0][0]']    
                                                                                                  
 conv4_block2_3_bn (BatchNormal  (None, 2, 2, 1024)  4096        ['conv4_block2_3_conv[0][0]']    
 ization) 

 n)                                                                                               
                                                                                                  
 conv4_block5_3_conv (Conv2D)   (None, 2, 2, 1024)   263168      ['conv4_block5_2_relu[0][0]']    
                                                                                                  
 conv4_block5_3_bn (BatchNormal  (None, 2, 2, 1024)  4096        ['conv4_block5_3_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block5_add (Add)         (None, 2, 2, 1024)   0           ['conv4_block4_out[0][0]',       
                                                                  'conv4_block5_3_bn[0][0]']      
                                                                                                  
 conv4_blo

 n)                                                                                               
                                                                                                  
 conv5_block2_3_conv (Conv2D)   (None, 1, 1, 2048)   1050624     ['conv5_block2_2_relu[0][0]']    
                                                                                                  
 conv5_block2_3_bn (BatchNormal  (None, 1, 1, 2048)  8192        ['conv5_block2_3_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv5_block2_add (Add)         (None, 1, 1, 2048)   0           ['conv5_block1_out[0][0]',       
                                                                  'conv5_block2_3_bn[0][0]']      
                                                                                                  
 conv5_blo

In [9]:
model_4.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=5, batch_size=32)

Epoch 1/5
1563/1563 [==============================] - 465s 293ms/step - loss: 1.2917 - accuracy: 0.5542 - val_loss: 1.1373 - val_accuracy: 0.6061
Epoch 2/5
1563/1563 [==============================] - 407s 260ms/step - loss: 1.0587 - accuracy: 0.6253 - val_loss: 1.1548 - val_accuracy: 0.5927
Epoch 3/5
1563/1563 [==============================] - 405s 259ms/step - loss: 0.9735 - accuracy: 0.6562 - val_loss: 1.1286 - val_accuracy: 0.6123
Epoch 4/5
1563/1563 [==============================] - 408s 261ms/step - loss: 0.9011 - accuracy: 0.6790 - val_loss: 1.1480 - val_accuracy: 0.6090
Epoch 5/5
1563/1563 [==============================] - 406s 260ms/step - loss: 0.8359 - accuracy: 0.7031 - val_loss: 1.1489 - val_accuracy: 0.6267


### Hasil Analisa Performa Kedua Model Pada Eksperimen Mengubah Variabel Jumlah Layer Dense

Kesimpulan:

<ul>
    <li>Nilai <code>'loss'</code> yang makin menurun/rendah nilainya dari epoch ke epoch selanjutnya menunjukkan kedua model semakin baik dalam memprediksi data training</li>
    <li>Nilai <code>'accuracy'</code> yang makin menaik/tinggi nilainya dari epoch ke epoch selanjutnya menunjukkan kedua model mampu memprediksi label data training dengan baik</li>
    <li>Nilai <code>'val_loss'</code> yang nilainya berubah naik turun secara tidak konsisten dari epoch ke epoch selanjutnya menunjukkan kedua model belum mampu menggeneralisasi data validation dengan baik</li>
    <li>Nilai <code>'val_accuracy'</code> yang semakin tinggi nilainya dari epoch ke epoch selanjutnya menunjukkan kedua model model mampu memprediksi data validation dengan baik, sehingga model memiliki kemampuan generalisasi yang baik</li>
    <li>Namun nilai <code>'val_loss'</code> lebih tinggi daripada nilai <code>'loss'</code> dan nilai <code>'val_accuracy'</code> lebih rendah dibandingkan nilai <code>'accuracy'</code>, ada kemungkinan kedua model mengalami overfitting pada data training</li>
    <li>Secara keseluruhan, pada model base ResNet-50, baik nilai <code>'loss'</code>, <code>'accuracy'</code>, <code>'val_loss'</code>, dan <code>'val_accuracy'</code>, semua nilainya ini lebih tinggi dibandingkan dengan semua nilai pada model base VGG-16</li>
    <li>Dan dibandingkan dengan performa awal kedua model secara keseluruhan, baik nilai <code>'loss'</code>, <code>'accuracy'</code>, <code>'val_loss'</code>, dan <code>'val_accuracy'</code>, semua nilainya ini lebih baik dalam memprediksi ketika menggunakan jumlah layer dense sebanyak 256 daripada ketika menggunakan 128 layer dense.</li>
</ul>

## Eksperimen Mengubah Variabel Optimizer

### Eksperimen 3 (Model Base VGG-16)

Model menggunakan pre-trained VGG-16, 128 layer dense neuron, optimizer <code>'sgd'</code>, 5 epoch, dan 32 batch size

In [10]:
# Membuat base model VGG-16
base_model_5 = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3)) # import model pre-trained VGG-16 yang telah dilath pada dataset ImageNet
base_model_5.trainable = False  # Membekukan lapisan pre-trained VGG-16

# Menambahkan lapisan baru setelah base model
x_5 = Flatten()(base_model_5.output)  # Flatten hasil feature extractor
x_5 = Dense(128, activation='relu')(x_5)  # Layer Dense dengan 128 neuron
output_5 = Dense(10, activation='softmax')(x_5)  # Layer output dengan 10 kelas (CIFAR-10)

# Membuat model akhir
model_5 = Model(inputs=base_model_5.input, outputs=output_5)

# Compile model
model_5.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Menampilkan ringkasan model
model_5.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 16, 16, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 16, 16, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 8, 8, 128)         0   

In [11]:
model_5.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=5, batch_size=32)

Epoch 1/5
1563/1563 [==============================] - 787s 503ms/step - loss: 1.7432 - accuracy: 0.4502 - val_loss: 1.4697 - val_accuracy: 0.5073
Epoch 2/5
1563/1563 [==============================] - 781s 500ms/step - loss: 1.3207 - accuracy: 0.5437 - val_loss: 1.3539 - val_accuracy: 0.5417
Epoch 3/5
1563/1563 [==============================] - 782s 500ms/step - loss: 1.2362 - accuracy: 0.5754 - val_loss: 1.3358 - val_accuracy: 0.5592
Epoch 4/5
1563/1563 [==============================] - 785s 502ms/step - loss: 1.1875 - accuracy: 0.5914 - val_loss: 1.3102 - val_accuracy: 0.5634
Epoch 5/5
1563/1563 [==============================] - 573s 367ms/step - loss: 1.1496 - accuracy: 0.6038 - val_loss: 1.3199 - val_accuracy: 0.5634


### Eksperimen 4 (Model Base ResNet-50)

Model menggunakan pre-trained ResNet-50, 128 layer dense neuron, optimizer <code>'sgd'</code>, 5 epoch, dan 32 batch size

In [12]:
# Membuat base model Resnet-50
base_model_6 = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3)) # import model pre-trained Resnet-50 yang telah dilath pada dataset ImageNet
base_model_6.trainable = False  # Membekukan lapisan pre-trained Resnet-50

# Menambahkan lapisan baru setelah base model
x_6 = Flatten()(base_model_6.output)  # Flatten hasil feature extractor
x_6 = Dense(128, activation='relu')(x_6)  # Layer Dense dengan 128 neuron
output_6 = Dense(10, activation='softmax')(x_6)  # Layer output dengan 10 kelas (CIFAR-10)

# Membuat model akhir
model_6 = Model(inputs=base_model_6.input, outputs=output_6)

# Compile model
model_6.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Menampilkan ringkasan model
model_6.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 38, 38, 3)    0           ['input_6[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 16, 16, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 16, 16, 64)   256         ['conv1_conv[0][0]']             
                                                                                            

                                                                                                  
 conv2_block3_2_bn (BatchNormal  (None, 8, 8, 64)    256         ['conv2_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv2_block3_2_relu (Activatio  (None, 8, 8, 64)    0           ['conv2_block3_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv2_block3_3_conv (Conv2D)   (None, 8, 8, 256)    16640       ['conv2_block3_2_relu[0][0]']    
                                                                                                  
 conv2_block3_3_bn (BatchNormal  (None, 8, 8, 256)   1024        ['conv2_block3_3_conv[0][0]']    
 ization) 

                                                                                                  
 conv3_block3_2_bn (BatchNormal  (None, 4, 4, 128)   512         ['conv3_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block3_2_relu (Activatio  (None, 4, 4, 128)   0           ['conv3_block3_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block3_3_conv (Conv2D)   (None, 4, 4, 512)    66048       ['conv3_block3_2_relu[0][0]']    
                                                                                                  
 conv3_block3_3_bn (BatchNormal  (None, 4, 4, 512)   2048        ['conv3_block3_3_conv[0][0]']    
 ization) 

                                                                                                  
 conv4_block2_2_bn (BatchNormal  (None, 2, 2, 256)   1024        ['conv4_block2_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block2_2_relu (Activatio  (None, 2, 2, 256)   0           ['conv4_block2_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block2_3_conv (Conv2D)   (None, 2, 2, 1024)   263168      ['conv4_block2_2_relu[0][0]']    
                                                                                                  
 conv4_block2_3_bn (BatchNormal  (None, 2, 2, 1024)  4096        ['conv4_block2_3_conv[0][0]']    
 ization) 

 n)                                                                                               
                                                                                                  
 conv4_block5_3_conv (Conv2D)   (None, 2, 2, 1024)   263168      ['conv4_block5_2_relu[0][0]']    
                                                                                                  
 conv4_block5_3_bn (BatchNormal  (None, 2, 2, 1024)  4096        ['conv4_block5_3_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block5_add (Add)         (None, 2, 2, 1024)   0           ['conv4_block4_out[0][0]',       
                                                                  'conv4_block5_3_bn[0][0]']      
                                                                                                  
 conv4_blo

 n)                                                                                               
                                                                                                  
 conv5_block2_3_conv (Conv2D)   (None, 1, 1, 2048)   1050624     ['conv5_block2_2_relu[0][0]']    
                                                                                                  
 conv5_block2_3_bn (BatchNormal  (None, 1, 1, 2048)  8192        ['conv5_block2_3_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv5_block2_add (Add)         (None, 1, 1, 2048)   0           ['conv5_block1_out[0][0]',       
                                                                  'conv5_block2_3_bn[0][0]']      
                                                                                                  
 conv5_blo

In [13]:
model_6.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=5, batch_size=32)

Epoch 1/5
1563/1563 [==============================] - 170s 106ms/step - loss: 1.3686 - accuracy: 0.5220 - val_loss: 1.2597 - val_accuracy: 0.5562
Epoch 2/5
1563/1563 [==============================] - 166s 106ms/step - loss: 1.1427 - accuracy: 0.5957 - val_loss: 1.1699 - val_accuracy: 0.5889
Epoch 3/5
1563/1563 [==============================] - 164s 105ms/step - loss: 1.0737 - accuracy: 0.6196 - val_loss: 1.1631 - val_accuracy: 0.6017
Epoch 4/5
1563/1563 [==============================] - 163s 104ms/step - loss: 1.0241 - accuracy: 0.6379 - val_loss: 1.1810 - val_accuracy: 0.5906
Epoch 5/5
1563/1563 [==============================] - 166s 106ms/step - loss: 0.9852 - accuracy: 0.6502 - val_loss: 1.1369 - val_accuracy: 0.6081


### Hasil Analisa Performa Kedua Model Pada Eksperimen Mengubah Variabel Optimizer

Kesimpulan:

<ul>
    <li>Nilai <code>'loss'</code> yang makin menurun/rendah nilainya dari epoch ke epoch selanjutnya menunjukkan kedua model semakin baik dalam memprediksi data training</li>
    <li>Nilai <code>'accuracy'</code> yang makin menaik/tinggi nilainya dari epoch ke epoch selanjutnya menunjukkan kedua model mampu memprediksi label data training dengan baik</li>
    <li>Nilai <code>'val_loss'</code> yang nilainya berubah naik turun secara tidak konsisten dari epoch ke epoch selanjutnya menunjukkan kedua model belum mampu menggeneralisasi data validation dengan baik</li>
    <li>Nilai <code>'val_accuracy'</code> yang semakin tinggi nilainya dari epoch ke epoch selanjutnya menunjukkan kedua model model mampu memprediksi data validation dengan baik, sehingga model memiliki kemampuan generalisasi yang baik</li>
    <li>Namun nilai <code>'val_loss'</code> lebih tinggi daripada nilai <code>'loss'</code> dan nilai <code>'val_accuracy'</code> lebih rendah dibandingkan nilai <code>'accuracy'</code>, ada kemungkinan kedua model mengalami overfitting pada data training</li>
    <li>Secara keseluruhan, pada model base ResNet-50, baik nilai <code>'loss'</code>, <code>'accuracy'</code>, <code>'val_loss'</code>, dan <code>'val_accuracy'</code>, semua nilainya ini lebih tinggi dibandingkan dengan semua nilai pada model base VGG-16</li>
    <li>Dan dibandingkan dengan performa awal kedua model secara keseluruhan, baik nilai <code>'loss'</code>, <code>'accuracy'</code>, <code>'val_loss'</code>, dan <code>'val_accuracy'</code>, semua nilainya ini lebih buruk dalam memprediksi ketika menggunakan optimizer <code>'sgd'</code> daripada ketika menggunakan optimizer <code>'adam'</code>.</li>
</ul>

## Eksperimen Mengubah Variabel Jumlah Epoch

### Eksperimen 5 (Model Base VGG-16)

Model menggunakan pre-trained VGG-16, 128 layer dense neuron, optimizer <code>'adam'</code>, 10 epoch, dan 32 batch size

In [14]:
# Membuat base model VGG-16
base_model_7 = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3)) # import model pre-trained VGG-16 yang telah dilath pada dataset ImageNet
base_model_7.trainable = False  # Membekukan lapisan pre-trained VGG-16

# Menambahkan lapisan baru setelah base model
x_7 = Flatten()(base_model_7.output)  # Flatten hasil feature extractor
x_7 = Dense(128, activation='relu')(x_7)  # Layer Dense dengan 128 neuron
output_7 = Dense(10, activation='softmax')(x_7)  # Layer output dengan 10 kelas (CIFAR-10)

# Membuat model akhir
model_7 = Model(inputs=base_model_7.input, outputs=output_7)

# Compile model
model_7.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Menampilkan ringkasan model
model_7.summary()

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 16, 16, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 16, 16, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 8, 8, 128)         0   

In [15]:
model_7.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=32)

Epoch 1/10
1563/1563 [==============================] - 252s 161ms/step - loss: 2.3327 - accuracy: 0.4656 - val_loss: 1.3433 - val_accuracy: 0.5463
Epoch 2/10
1563/1563 [==============================] - 249s 159ms/step - loss: 1.2148 - accuracy: 0.5800 - val_loss: 1.2331 - val_accuracy: 0.5847
Epoch 3/10
1563/1563 [==============================] - 252s 161ms/step - loss: 1.1255 - accuracy: 0.6148 - val_loss: 1.2132 - val_accuracy: 0.5892
Epoch 4/10
1563/1563 [==============================] - 251s 161ms/step - loss: 1.0800 - accuracy: 0.6289 - val_loss: 1.2005 - val_accuracy: 0.5956
Epoch 5/10
1563/1563 [==============================] - 253s 162ms/step - loss: 1.0444 - accuracy: 0.6438 - val_loss: 1.2321 - val_accuracy: 0.5953
Epoch 6/10
1563/1563 [==============================] - 251s 161ms/step - loss: 1.0066 - accuracy: 0.6543 - val_loss: 1.2278 - val_accuracy: 0.6056
Epoch 7/10
1563/1563 [==============================] - 257s 164ms/step - loss: 0.9760 - accuracy: 0.6643 - val_

### Eksperimen 6 (Model Base ResNet-50)

Model menggunakan pre-trained ResNet-50, 128 layer dense neuron, optimizer <code>'adam'</code>, 10 epoch, dan 32 batch size

In [16]:
# Membuat base model Resnet-50
base_model_8 = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3)) # import model pre-trained Resnet-50 yang telah dilath pada dataset ImageNet
base_model_8.trainable = False  # Membekukan lapisan pre-trained Resnet-50

# Menambahkan lapisan baru setelah base model
x_8 = Flatten()(base_model_8.output)  # Flatten hasil feature extractor
x_8 = Dense(128, activation='relu')(x_8)  # Layer Dense dengan 128 neuron
output_8 = Dense(10, activation='softmax')(x_8)  # Layer output dengan 10 kelas (CIFAR-10)

# Membuat model akhir
model_8 = Model(inputs=base_model_8.input, outputs=output_8)

# Compile model
model_8.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Menampilkan ringkasan model
model_8.summary()

Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_8 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 38, 38, 3)    0           ['input_8[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 16, 16, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 16, 16, 64)   256         ['conv1_conv[0][0]']             
                                                                                            

                                                                                                  
 conv2_block3_2_bn (BatchNormal  (None, 8, 8, 64)    256         ['conv2_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv2_block3_2_relu (Activatio  (None, 8, 8, 64)    0           ['conv2_block3_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv2_block3_3_conv (Conv2D)   (None, 8, 8, 256)    16640       ['conv2_block3_2_relu[0][0]']    
                                                                                                  
 conv2_block3_3_bn (BatchNormal  (None, 8, 8, 256)   1024        ['conv2_block3_3_conv[0][0]']    
 ization) 

                                                                                                  
 conv3_block3_2_bn (BatchNormal  (None, 4, 4, 128)   512         ['conv3_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block3_2_relu (Activatio  (None, 4, 4, 128)   0           ['conv3_block3_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block3_3_conv (Conv2D)   (None, 4, 4, 512)    66048       ['conv3_block3_2_relu[0][0]']    
                                                                                                  
 conv3_block3_3_bn (BatchNormal  (None, 4, 4, 512)   2048        ['conv3_block3_3_conv[0][0]']    
 ization) 

                                                                                                  
 conv4_block2_2_bn (BatchNormal  (None, 2, 2, 256)   1024        ['conv4_block2_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block2_2_relu (Activatio  (None, 2, 2, 256)   0           ['conv4_block2_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block2_3_conv (Conv2D)   (None, 2, 2, 1024)   263168      ['conv4_block2_2_relu[0][0]']    
                                                                                                  
 conv4_block2_3_bn (BatchNormal  (None, 2, 2, 1024)  4096        ['conv4_block2_3_conv[0][0]']    
 ization) 

 n)                                                                                               
                                                                                                  
 conv4_block5_3_conv (Conv2D)   (None, 2, 2, 1024)   263168      ['conv4_block5_2_relu[0][0]']    
                                                                                                  
 conv4_block5_3_bn (BatchNormal  (None, 2, 2, 1024)  4096        ['conv4_block5_3_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block5_add (Add)         (None, 2, 2, 1024)   0           ['conv4_block4_out[0][0]',       
                                                                  'conv4_block5_3_bn[0][0]']      
                                                                                                  
 conv4_blo

 n)                                                                                               
                                                                                                  
 conv5_block2_3_conv (Conv2D)   (None, 1, 1, 2048)   1050624     ['conv5_block2_2_relu[0][0]']    
                                                                                                  
 conv5_block2_3_bn (BatchNormal  (None, 1, 1, 2048)  8192        ['conv5_block2_3_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv5_block2_add (Add)         (None, 1, 1, 2048)   0           ['conv5_block1_out[0][0]',       
                                                                  'conv5_block2_3_bn[0][0]']      
                                                                                                  
 conv5_blo

In [17]:
model_8.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=32)

Epoch 1/10
1563/1563 [==============================] - 207s 131ms/step - loss: 1.3023 - accuracy: 0.5496 - val_loss: 1.1872 - val_accuracy: 0.5869
Epoch 2/10
1563/1563 [==============================] - 186s 119ms/step - loss: 1.0697 - accuracy: 0.6257 - val_loss: 1.1543 - val_accuracy: 0.5993
Epoch 3/10
1563/1563 [==============================] - 165s 106ms/step - loss: 0.9932 - accuracy: 0.6496 - val_loss: 1.1171 - val_accuracy: 0.6165
Epoch 4/10
1563/1563 [==============================] - 167s 107ms/step - loss: 0.9335 - accuracy: 0.6690 - val_loss: 1.1175 - val_accuracy: 0.6194
Epoch 5/10
1563/1563 [==============================] - 164s 105ms/step - loss: 0.8830 - accuracy: 0.6869 - val_loss: 1.1280 - val_accuracy: 0.6143
Epoch 6/10
1563/1563 [==============================] - 170s 109ms/step - loss: 0.8347 - accuracy: 0.7039 - val_loss: 1.1578 - val_accuracy: 0.6209
Epoch 7/10
1563/1563 [==============================] - 166s 106ms/step - loss: 0.7830 - accuracy: 0.7229 - val_

### Hasil Analisa Performa Kedua Model Pada Eksperimen Mengubah Variabel Jumlah Epoch

Kesimpulan:

<ul>
    <li>Nilai <code>'loss'</code> yang makin menurun/rendah nilainya dari epoch ke epoch selanjutnya menunjukkan kedua model semakin baik dalam memprediksi data training</li>
    <li>Nilai <code>'accuracy'</code> yang makin menaik/tinggi nilainya dari epoch ke epoch selanjutnya menunjukkan kedua model mampu memprediksi label data training dengan baik</li>
    <li>Nilai <code>'val_loss'</code> yang nilainya berubah naik turun secara tidak konsisten dari epoch ke epoch selanjutnya menunjukkan kedua model belum mampu menggeneralisasi data validation dengan baik</li>
    <li>Nilai <code>'val_accuracy'</code> yang semakin tinggi nilainya dari epoch ke epoch selanjutnya menunjukkan kedua model model mampu memprediksi data validation dengan baik, sehingga model memiliki kemampuan generalisasi yang baik</li>
    <li>Namun nilai <code>'val_loss'</code> lebih tinggi daripada nilai <code>'loss'</code> dan nilai <code>'val_accuracy'</code> lebih rendah dibandingkan nilai <code>'accuracy'</code>, ada kemungkinan kedua model mengalami overfitting pada data training</li>
    <li>Secara keseluruhan, pada model base ResNet-50, baik nilai <code>'loss'</code>, <code>'accuracy'</code>, <code>'val_loss'</code>, dan <code>'val_accuracy'</code>, semua nilainya ini lebih tinggi dibandingkan dengan semua nilai pada model base VGG-16</li>
    <li>Dan dibandingkan dengan performa awal kedua model secara keseluruhan, baik nilai <code>'loss'</code>, <code>'accuracy'</code>, <code>'val_loss'</code>, dan <code>'val_accuracy'</code>, semua nilainya ini jauh lebih baik dalam memprediksi ketika menggunakan epoch sebanyak 10 daripada ketika menggunakan 5 epoch saja ataupun jika hanya mengubah jumlah layer dense yang lebih besar seperti pada eksperimen sebelumnya.</li>
</ul>

## Eksperimen Mengubah Variabel Jumlah Batch Size

### Eksperimen 7 (Model Base VGG-16)

Model menggunakan pre-trained VGG-16, 128 layer dense neuron, optimizer <code>'adam'</code>, 5 epoch, dan 64 batch size

In [18]:
# Membuat base model VGG-16
base_model_9 = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3)) # import model pre-trained VGG-16 yang telah dilath pada dataset ImageNet
base_model_9.trainable = False  # Membekukan lapisan pre-trained VGG-16

# Menambahkan lapisan baru setelah base model
x_9 = Flatten()(base_model_9.output)  # Flatten hasil feature extractor
x_9 = Dense(128, activation='relu')(x_9)  # Layer Dense dengan 128 neuron
output_9 = Dense(10, activation='softmax')(x_9)  # Layer output dengan 10 kelas (CIFAR-10)

# Membuat model akhir
model_9 = Model(inputs=base_model_9.input, outputs=output_9)

# Compile model
model_9.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Menampilkan ringkasan model
model_9.summary()

Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 16, 16, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 16, 16, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 8, 8, 128)         0   

In [19]:
model_9.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=5, batch_size=64)

Epoch 1/5
782/782 [==============================] - 225s 286ms/step - loss: 2.6192 - accuracy: 0.4452 - val_loss: 1.4729 - val_accuracy: 0.5055
Epoch 2/5
782/782 [==============================] - 224s 286ms/step - loss: 1.2759 - accuracy: 0.5615 - val_loss: 1.3015 - val_accuracy: 0.5568
Epoch 3/5
782/782 [==============================] - 221s 283ms/step - loss: 1.1310 - accuracy: 0.6068 - val_loss: 1.2408 - val_accuracy: 0.5840
Epoch 4/5
782/782 [==============================] - 222s 284ms/step - loss: 1.0579 - accuracy: 0.6314 - val_loss: 1.2239 - val_accuracy: 0.5945
Epoch 5/5
782/782 [==============================] - 223s 285ms/step - loss: 1.0095 - accuracy: 0.6487 - val_loss: 1.2230 - val_accuracy: 0.5920


### Eksperimen 8 (Model Base ResNet-50)

Model menggunakan pre-trained ResNet-50, 128 layer dense neuron, optimizer <code>'adam'</code>, 10 epoch, dan 64 batch size

In [20]:
# Membuat base model Resnet-50
base_model_10 = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3)) # import model pre-trained Resnet-50 yang telah dilath pada dataset ImageNet
base_model_10.trainable = False  # Membekukan lapisan pre-trained Resnet-50

# Menambahkan lapisan baru setelah base model
x_10 = Flatten()(base_model_10.output)  # Flatten hasil feature extractor
x_10 = Dense(128, activation='relu')(x_10)  # Layer Dense dengan 128 neuron
output_10 = Dense(10, activation='softmax')(x_10)  # Layer output dengan 10 kelas (CIFAR-10)

# Membuat model akhir
model_10 = Model(inputs=base_model_10.input, outputs=output_10)

# Compile model
model_10.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Menampilkan ringkasan model
model_10.summary()

Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_10 (InputLayer)          [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 38, 38, 3)    0           ['input_10[0][0]']               
                                                                                                  
 conv1_conv (Conv2D)            (None, 16, 16, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 16, 16, 64)   256         ['conv1_conv[0][0]']             
                                                                                            

                                                                                                  
 conv2_block3_2_bn (BatchNormal  (None, 8, 8, 64)    256         ['conv2_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv2_block3_2_relu (Activatio  (None, 8, 8, 64)    0           ['conv2_block3_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv2_block3_3_conv (Conv2D)   (None, 8, 8, 256)    16640       ['conv2_block3_2_relu[0][0]']    
                                                                                                  
 conv2_block3_3_bn (BatchNormal  (None, 8, 8, 256)   1024        ['conv2_block3_3_conv[0][0]']    
 ization) 

                                                                                                  
 conv3_block3_2_bn (BatchNormal  (None, 4, 4, 128)   512         ['conv3_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block3_2_relu (Activatio  (None, 4, 4, 128)   0           ['conv3_block3_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block3_3_conv (Conv2D)   (None, 4, 4, 512)    66048       ['conv3_block3_2_relu[0][0]']    
                                                                                                  
 conv3_block3_3_bn (BatchNormal  (None, 4, 4, 512)   2048        ['conv3_block3_3_conv[0][0]']    
 ization) 

                                                                                                  
 conv4_block2_2_bn (BatchNormal  (None, 2, 2, 256)   1024        ['conv4_block2_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block2_2_relu (Activatio  (None, 2, 2, 256)   0           ['conv4_block2_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block2_3_conv (Conv2D)   (None, 2, 2, 1024)   263168      ['conv4_block2_2_relu[0][0]']    
                                                                                                  
 conv4_block2_3_bn (BatchNormal  (None, 2, 2, 1024)  4096        ['conv4_block2_3_conv[0][0]']    
 ization) 

 n)                                                                                               
                                                                                                  
 conv4_block5_3_conv (Conv2D)   (None, 2, 2, 1024)   263168      ['conv4_block5_2_relu[0][0]']    
                                                                                                  
 conv4_block5_3_bn (BatchNormal  (None, 2, 2, 1024)  4096        ['conv4_block5_3_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block5_add (Add)         (None, 2, 2, 1024)   0           ['conv4_block4_out[0][0]',       
                                                                  'conv4_block5_3_bn[0][0]']      
                                                                                                  
 conv4_blo

 n)                                                                                               
                                                                                                  
 conv5_block2_3_conv (Conv2D)   (None, 1, 1, 2048)   1050624     ['conv5_block2_2_relu[0][0]']    
                                                                                                  
 conv5_block2_3_bn (BatchNormal  (None, 1, 1, 2048)  8192        ['conv5_block2_3_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv5_block2_add (Add)         (None, 1, 1, 2048)   0           ['conv5_block1_out[0][0]',       
                                                                  'conv5_block2_3_bn[0][0]']      
                                                                                                  
 conv5_blo

In [21]:
model_10.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=5, batch_size=64)

Epoch 1/5
782/782 [==============================] - 139s 175ms/step - loss: 1.3177 - accuracy: 0.5448 - val_loss: 1.1678 - val_accuracy: 0.5956
Epoch 2/5
782/782 [==============================] - 138s 176ms/step - loss: 1.0620 - accuracy: 0.6243 - val_loss: 1.1532 - val_accuracy: 0.6014
Epoch 3/5
782/782 [==============================] - 138s 177ms/step - loss: 0.9740 - accuracy: 0.6542 - val_loss: 1.1141 - val_accuracy: 0.6198
Epoch 4/5
782/782 [==============================] - 139s 178ms/step - loss: 0.9087 - accuracy: 0.6789 - val_loss: 1.1293 - val_accuracy: 0.6153
Epoch 5/5
782/782 [==============================] - 136s 174ms/step - loss: 0.8525 - accuracy: 0.6956 - val_loss: 1.1481 - val_accuracy: 0.6103


### Hasil Analisa Performa Kedua Model Pada Eksperimen Mengubah Variabel Jumlah Epoch

Kesimpulan:

<ul>
    <li>Nilai <code>'loss'</code> yang makin menurun/rendah nilainya dari epoch ke epoch selanjutnya menunjukkan kedua model semakin baik dalam memprediksi data training</li>
    <li>Nilai <code>'accuracy'</code> yang makin menaik/tinggi nilainya dari epoch ke epoch selanjutnya menunjukkan kedua model mampu memprediksi label data training dengan baik</li>
    <li>Nilai <code>'val_loss'</code> yang nilainya berubah naik turun secara tidak konsisten dari epoch ke epoch selanjutnya menunjukkan kedua model belum mampu menggeneralisasi data validation dengan baik</li>
    <li>Nilai <code>'val_accuracy'</code> yang semakin tinggi nilainya dari epoch ke epoch selanjutnya menunjukkan kedua model model mampu memprediksi data validation dengan baik, sehingga model memiliki kemampuan generalisasi yang baik</li>
    <li>Namun nilai <code>'val_loss'</code> lebih tinggi daripada nilai <code>'loss'</code> dan nilai <code>'val_accuracy'</code> lebih rendah dibandingkan nilai <code>'accuracy'</code>, ada kemungkinan kedua model mengalami overfitting pada data training</li>
    <li>Secara keseluruhan, pada model base ResNet-50, baik nilai <code>'loss'</code>, <code>'accuracy'</code>, <code>'val_loss'</code>, dan <code>'val_accuracy'</code>, semua nilainya ini lebih tinggi dibandingkan dengan semua nilai pada model base VGG-16</li>
    <li>Dan dibandingkan dengan performa awal kedua model secara keseluruhan, baik nilai <code>'loss'</code>, <code>'accuracy'</code>, <code>'val_loss'</code>, dan <code>'val_accuracy'</code>, semua nilainya ini tidak jauh berbeda karena hanya sedikit meningkat dalam memprediksi ketika menggunakan jumlah batch size sebanyak 64 daripada ketika menggunakan 32 batch size.</li>
</ul>

## Perbandingan Hasil Analisa Performa Kedua Model Dari Semua Eksperimen Dengan Performa Awal Kedua Model

Kesimpulan:

<ul>
    <li>Nilai <code>'loss'</code> yang makin menurun/rendah nilainya dari epoch ke epoch selanjutnya menunjukkan kedua model semakin baik dalam memprediksi data training</li>
    <li>Nilai <code>'accuracy'</code> yang makin menaik/tinggi nilainya dari epoch ke epoch selanjutnya menunjukkan kedua model mampu memprediksi label data training dengan baik</li>
    <li>Nilai <code>'val_loss'</code> yang nilainya berubah naik turun secara tidak konsisten dari epoch ke epoch selanjutnya menunjukkan kedua model belum mampu menggeneralisasi data validation dengan baik</li>
    <li>Nilai <code>'val_accuracy'</code> yang semakin tinggi nilainya dari epoch ke epoch selanjutnya menunjukkan kedua model model mampu memprediksi data validation dengan baik, sehingga model memiliki kemampuan generalisasi yang baik</li>
    <li>Namun nilai <code>'val_loss'</code> lebih tinggi daripada nilai <code>'loss'</code> dan nilai <code>'val_accuracy'</code> lebih rendah dibandingkan nilai <code>'accuracy'</code>, ada kemungkinan kedua model mengalami overfitting pada data training</li>
    <li>Secara keseluruhan, pada model base ResNet-50, baik nilai <code>'loss'</code>, <code>'accuracy'</code>, <code>'val_loss'</code>, dan <code>'val_accuracy'</code>, semua nilainya ini lebih tinggi dibandingkan dengan semua nilai pada model base VGG-16</li>
    <li>Dan dibandingkan dengan performa awal kedua model secara keseluruhan, baik nilai <code>'loss'</code>, <code>'accuracy'</code>, <code>'val_loss'</code>, dan <code>'val_accuracy'</code>, semua nilainya ini sangat baik dalam memprediksi ketika mengubah jumlah epoch naik menjadi sebanyak 10. Meski dengan mengubah jumlah layer dense naik menjadi sebanyak 256 ataupun mengubah batch size naik sebanyak 64, semua pengubahan itu masih jauh lebih tinggi nilai akurasi dan nilai loss nya ketika dengan hanya mengubah variabel epoch yang lebih tinggi.</li>
    <li>Sementara itu, pengubahan variabel optimizer 'sgd' tidak memberikan hasil prediksi yang lebih baik. Sehingga dalam hal ini, penggunaan optimizer 'adam' masih lebih baik dalam memberikan model prediksi yang baik.</li>
    <li>Karena sebelumnya hanya melakukan eksperimen dengan hanya mengubah 1 variabel saja, mungkin saja jika dilakukan pengubahan beberapa variabel (jumlah layer dense, epoch, dan batch size) secara bersamaan bisa saja meningkatkan akurasi yang lebih tinggi. Sehingga dilakukan eksperimen terakhir setelah ini untuk melihat apakah perkiraan tersebut adalah benar atau tidak.</li>
</ul>

## Eksperimen Mengubah Variabel Jumlah Layer Dense, Epoch, dan Batch Size

### Eksperimen 9 (Model Base VGG-16)

Model menggunakan pre-trained VGG-16, 256 layer dense neuron, optimizer <code>'adam'</code>, 10 epoch, dan 64 batch size

In [22]:
# Membuat base model VGG-16
base_model_11 = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3)) # import model pre-trained VGG-16 yang telah dilath pada dataset ImageNet
base_model_11.trainable = False  # Membekukan lapisan pre-trained VGG-16

# Menambahkan lapisan baru setelah base model
x_11 = Flatten()(base_model_11.output)  # Flatten hasil feature extractor
x_11 = Dense(256, activation='relu')(x_11)  # Layer Dense dengan 256 neuron
output_11 = Dense(10, activation='softmax')(x_11)  # Layer output dengan 10 kelas (CIFAR-10)

# Membuat model akhir
model_11 = Model(inputs=base_model_11.input, outputs=output_11)

# Compile model
model_11.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Menampilkan ringkasan model
model_11.summary()

Model: "model_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, 32, 32, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 16, 16, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 16, 16, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 8, 8, 128)         0  

In [23]:
model_11.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
782/782 [==============================] - 303s 386ms/step - loss: 2.6401 - accuracy: 0.4761 - val_loss: 1.3764 - val_accuracy: 0.5469
Epoch 2/10
782/782 [==============================] - 217s 277ms/step - loss: 1.1780 - accuracy: 0.5958 - val_loss: 1.2406 - val_accuracy: 0.5861
Epoch 3/10
782/782 [==============================] - 224s 287ms/step - loss: 1.0301 - accuracy: 0.6415 - val_loss: 1.2404 - val_accuracy: 0.5896
Epoch 4/10
782/782 [==============================] - 239s 305ms/step - loss: 0.9573 - accuracy: 0.6675 - val_loss: 1.2425 - val_accuracy: 0.5947
Epoch 5/10
782/782 [==============================] - 254s 324ms/step - loss: 0.8964 - accuracy: 0.6865 - val_loss: 1.2545 - val_accuracy: 0.6002
Epoch 6/10
782/782 [==============================] - 247s 315ms/step - loss: 0.8440 - accuracy: 0.7038 - val_loss: 1.3071 - val_accuracy: 0.5914
Epoch 7/10
782/782 [==============================] - 229s 293ms/step - loss: 0.7846 - accuracy: 0.7255 - val_loss: 1.3612 -

### Eksperimen 10 (Model Base ResNet-50)

Model menggunakan pre-trained ResNet-50, 256 layer dense neuron, optimizer <code>'adam'</code>, 10 epoch, dan 64 batch size

In [24]:
# Membuat base model VGG-16
base_model_12 = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3)) # import model pre-trained ResNet-50 yang telah dilath pada dataset ImageNet
base_model_12.trainable = False  # Membekukan lapisan pre-trained VGG-16

# Menambahkan lapisan baru setelah base model
x_12 = Flatten()(base_model_12.output)  # Flatten hasil feature extractor
x_12 = Dense(256, activation='relu')(x_12)  # Layer Dense dengan 256 neuron
output_12 = Dense(10, activation='softmax')(x_12)  # Layer output dengan 10 kelas (CIFAR-10)

# Membuat model akhir
model_12 = Model(inputs=base_model_12.input, outputs=output_12)

# Compile model
model_12.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Menampilkan ringkasan model
model_12.summary()

Model: "model_11"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_12 (InputLayer)          [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 38, 38, 3)    0           ['input_12[0][0]']               
                                                                                                  
 conv1_conv (Conv2D)            (None, 16, 16, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 16, 16, 64)   256         ['conv1_conv[0][0]']             
                                                                                           

                                                                                                  
 conv2_block3_2_bn (BatchNormal  (None, 8, 8, 64)    256         ['conv2_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv2_block3_2_relu (Activatio  (None, 8, 8, 64)    0           ['conv2_block3_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv2_block3_3_conv (Conv2D)   (None, 8, 8, 256)    16640       ['conv2_block3_2_relu[0][0]']    
                                                                                                  
 conv2_block3_3_bn (BatchNormal  (None, 8, 8, 256)   1024        ['conv2_block3_3_conv[0][0]']    
 ization) 

                                                                                                  
 conv3_block3_2_bn (BatchNormal  (None, 4, 4, 128)   512         ['conv3_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block3_2_relu (Activatio  (None, 4, 4, 128)   0           ['conv3_block3_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block3_3_conv (Conv2D)   (None, 4, 4, 512)    66048       ['conv3_block3_2_relu[0][0]']    
                                                                                                  
 conv3_block3_3_bn (BatchNormal  (None, 4, 4, 512)   2048        ['conv3_block3_3_conv[0][0]']    
 ization) 

                                                                                                  
 conv4_block2_2_bn (BatchNormal  (None, 2, 2, 256)   1024        ['conv4_block2_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block2_2_relu (Activatio  (None, 2, 2, 256)   0           ['conv4_block2_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block2_3_conv (Conv2D)   (None, 2, 2, 1024)   263168      ['conv4_block2_2_relu[0][0]']    
                                                                                                  
 conv4_block2_3_bn (BatchNormal  (None, 2, 2, 1024)  4096        ['conv4_block2_3_conv[0][0]']    
 ization) 

 n)                                                                                               
                                                                                                  
 conv4_block5_3_conv (Conv2D)   (None, 2, 2, 1024)   263168      ['conv4_block5_2_relu[0][0]']    
                                                                                                  
 conv4_block5_3_bn (BatchNormal  (None, 2, 2, 1024)  4096        ['conv4_block5_3_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block5_add (Add)         (None, 2, 2, 1024)   0           ['conv4_block4_out[0][0]',       
                                                                  'conv4_block5_3_bn[0][0]']      
                                                                                                  
 conv4_blo

 n)                                                                                               
                                                                                                  
 conv5_block2_3_conv (Conv2D)   (None, 1, 1, 2048)   1050624     ['conv5_block2_2_relu[0][0]']    
                                                                                                  
 conv5_block2_3_bn (BatchNormal  (None, 1, 1, 2048)  8192        ['conv5_block2_3_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv5_block2_add (Add)         (None, 1, 1, 2048)   0           ['conv5_block1_out[0][0]',       
                                                                  'conv5_block2_3_bn[0][0]']      
                                                                                                  
 conv5_blo

In [25]:
model_12.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
782/782 [==============================] - 148s 186ms/step - loss: 1.2955 - accuracy: 0.5560 - val_loss: 1.1333 - val_accuracy: 0.5995
Epoch 2/10
782/782 [==============================] - 142s 181ms/step - loss: 1.0346 - accuracy: 0.6344 - val_loss: 1.1143 - val_accuracy: 0.6165
Epoch 3/10
782/782 [==============================] - 141s 181ms/step - loss: 0.9410 - accuracy: 0.6674 - val_loss: 1.1046 - val_accuracy: 0.6207
Epoch 4/10
782/782 [==============================] - 142s 182ms/step - loss: 0.8651 - accuracy: 0.6923 - val_loss: 1.1384 - val_accuracy: 0.6151
Epoch 5/10
782/782 [==============================] - 141s 180ms/step - loss: 0.7853 - accuracy: 0.7214 - val_loss: 1.1848 - val_accuracy: 0.6051
Epoch 6/10
782/782 [==============================] - 142s 182ms/step - loss: 0.7165 - accuracy: 0.7435 - val_loss: 1.1913 - val_accuracy: 0.6161
Epoch 7/10
782/782 [==============================] - 147s 188ms/step - loss: 0.6452 - accuracy: 0.7709 - val_loss: 1.2754 -

## Kesimpulan Akhir Hasil Analisa Performa Dengan Pengubahan 3 Variabel Secara Bersamaan Pada Kedua Performa

Berbeda saat ketika melakukan ekperimen dengan hanya mengubah 1 variabel saja, dan ketika dilakukan pengubahan 3 variabel (jumlah layer dense, epoch, dan batch size) sekaligus, ternyata eksperimen dengan model ini memberikan hasil performa yang jauh lebih baik dibandingkan dengan hanya mengubah 1 varibel masing-masing tersebut karena meningkatkan nilai akurasi yang lebih tinggi secara drastis. Sehingga bisa disimpulkan bahwa semakin dalam atau banyak jumlah dense layer, dan semakin banyak jumlah pelatihan (epoch dan batch size) yang digunakan maka hasil prediksi pun akan semakin jauh lebih baik hasil yang didapatkan.